EVALUATION METRICS

In [25]:
import json
import math
import os
import re
import collections
from collections import defaultdict
from array import array
import numpy as np
import pandas as pd
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))  # one level up from part_2
sys.path.append(project_root)
from part_2.indexing_evaluation import load_processed_docs,create_index_tfidf,search_tf_idf

In [26]:
def precision_k(y_true,y_score,k=10):
    order=np.argsort(np.asarray(y_score))[::-1]
    y_true= np.take(np.asarray(y_true),order)
    k=min(k,len(y_true)) #handle if k> len(y_true)
    return np.sum(y_true[:k])/k if k>0 else 0.0

In [27]:
def recall_k(y_true,y_score,k=10):
    order=np.argsort(np.asarray(y_score))[::-1]
    y_true=np.take(np.asarray(y_true),order)
    relevenat=np.sum(y_true[:k])
    total_rel=np.sum(np.asarray(y_true))
    return relevenat/total_rel if total_rel>0 else 0.0

In [28]:
def f1_k(y_true,y_score,k=10):
    prec=precision_k(y_true,y_score,k)
    rec=recall_k(y_true,y_score,k)
    return (2*prec*rec)/(prec+rec) if (prec+rec)>0 else 0.0

In [29]:
def average_precision(y_true,y_score,k=10):
    order=np.argsort(y_score)[::-1]
    y_true=np.take(np.asarray(y_true),order)
    prec_list=[]
    num_relevant=0
    for i in range(min(k,len(order))):
        if y_true[i]==1:
            num_relevant +=1
            prec_list.append(num_relevant/(i+1))
    return np.sum(prec_list)/num_relevant if num_relevant>0 else 0.0

In [30]:
def map_k(dt,k):
    avp = []
    for q in dt["query_id"].unique():  
        curr_data = dt[dt["query_id"] == q] 
        y_true=curr_data["labels"].values
        y_score=curr_data["predicted_relevance"].values
        avp.append(average_precision(y_true,y_score,k)) 
    return np.sum(avp) / len(avp), avp  

In [31]:
def rr_k(y_true,y_score,k=10):
    order=np.argsort(np.asarray(y_score))[::-1]
    y_true= np.take(np.asarray(y_true),order)[:k]
    if np.sum(y_true)==0: 
        return 0
    return 1/(np.argmax(y_true)+1)

In [32]:
def dcg_k(y_true,y_score,k=10):
    order=np.argsort(y_score)[::-1]
    y_true=np.take(y_true,order[:k])
    gain= 2**y_true-1
    discounts=np.log2(np.arange(len(y_true))+2) #+2 is added because log2(1) is 0
    return np.sum(gain/discounts)

In [33]:
def ndcg_k(y_true,y_score,k=10):
    dcg= dcg_k(y_true,y_score,k)
    idcg=dcg_k(y_true,y_true,k)
    if not idcg:
        return 0
    return round(dcg/idcg,4)


In [34]:
'''
if __name__=="__main__":
    docs=load_processed_docs()
    
    index,tf,df_counts,idf,title_index=create_index_tfidf(docs)
    
    validation_path='../data/validation_labels.csv'
    validation_df=pd.read_csv(validation_path)

    queries={1:"women full sleeve sweatshirt cotton",
             2:"men slim jeans blue"}
    
    result=[]

    validation_df["predicted_relevance"]
    for q, query in queries.items():
        retrieve_pid=search_tf_idf(query,index,tf,idf,title_index)
        rank_score = {pid: len(retrieve_pid) - i for i, pid in enumerate(retrieve_pid)}
        validation_df.loc[validation_df["query_id"] == q, "predicted_relevance"] = validation_df.loc[validation_df["query_id"] == q, "pid"].apply(lambda pid: rank_score.get(pid, 0))

    for i in queries.keys():
        query_doc=validation_df[validation_df["query_id"]==i]
        y_true=query_doc["labels"].values
        y_score=query_doc["predicted_relevance"].values

        result.append({
            "query_id":i,
            "Precision@5": round(precision_k(y_true,y_score,5),3),
            "Recalln@5": round(recall_k(y_true,y_score,5),3),
            "AP@5": round(average_precision(y_true,y_score,5),3),
            "F1@5": round(f1_k(y_true,y_score,5),3),
            "MRR": round(rr_k(y_true,y_score,5),3),
            "NDCG": round(ndcg_k(y_true,y_score,5),3),
        })
    results_df=pd.DataFrame(result)
    print(results_df)

    mean_avp,avp_list=map_k(validation_df,k=5)
    print("MAP:",round(mean_avp,3))
    
'''
    

'\nif __name__=="__main__":\n    docs=load_processed_docs()\n    \n    index,tf,df_counts,idf,title_index=create_index_tfidf(docs)\n    \n    validation_path=\'../data/validation_labels.csv\'\n    validation_df=pd.read_csv(validation_path)\n\n    queries={1:"women full sleeve sweatshirt cotton",\n             2:"men slim jeans blue"}\n    \n    result=[]\n\n    validation_df["predicted_relevance"]\n    for q, query in queries.items():\n        retrieve_pid=search_tf_idf(query,index,tf,idf,title_index)\n        rank_score = {pid: len(retrieve_pid) - i for i, pid in enumerate(retrieve_pid)}\n        validation_df.loc[validation_df["query_id"] == q, "predicted_relevance"] = validation_df.loc[validation_df["query_id"] == q, "pid"].apply(lambda pid: rank_score.get(pid, 0))\n\n    for i in queries.keys():\n        query_doc=validation_df[validation_df["query_id"]==i]\n        y_true=query_doc["labels"].values\n        y_score=query_doc["predicted_relevance"].values\n\n        result.append({